# Interpretabilidad de modelos
> Arrojando luz a las cajas negras.

#### INDICE

- **I. Introducción.**
- **II. Preparando los datos.**
- **III. EDA automático.**
- **IV. Entrenamiento del modelo.**
- **V. Local Interpretable Model-agnostic Explanations.**
- **Recursos.**

## I. Introducción

En machine learning para poder obtener un modelo fiable una de las cosas que se realizan es la validación cruzada. Esto nos ayuda a ver una visión a alto nivel de cómo es el modelo, pero ¿cómo este modelo ha aprendido? ¿podemos ver el aprendizaje de este y que lleva a llamar falso algo y verdadero otra cosa?

¿Cuáles son las características que debe tener una herramienta que explique modelos?

- Interpretable: Debe proporcionar una comprensión cualitativa entre las variables de entrada y la respuesta. Debe ser fácil de entender.

- Fidelidad local: Es posible que una explicación no sea completamente fiel a menos que sea la descripción completa del propio modelo. Dicho esto, debe reproducir el comportamiento del modelo en las proximidades de la instancia que se predice. O lo que es lo mismo, pequeñas varaiciones en las features deben de dar el mismo resultado.

- Agnóstico al modelo: No debe hacer suposiciones sobre modelos, es decir el explicador es transparente para el modelo.

- Perspectiva global: El explicador debe explicar un conjunto representativo al usuario para que éste tenga una intuición global del modelo.

## II. Preparando los datos

In [14]:
import lightgbm as lgb
import lime
import lime.lime_tabular
import numpy as np
import pandas as pd
import sweetviz as sv
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv("./datos/data.csv")
df.drop("Unnamed: 32", axis=1, inplace=True)

In [3]:
# Cambiamos los valores de B => 0 y M => 1
df["diagnosis"] = df["diagnosis"].apply(lambda x: 0 if x == "B" else 1)  

Para tener un set de train y test valanceado vamos a separarlo con la misma proporción de cada etiqueta en ambos sets

In [4]:
target_column = "diagnosis"
featu_columns = list(set(df.columns) - {target_column})

In [5]:
feat_data = df[featu_columns].values
targ_data = df[target_column].values

In [6]:
feat_train, feat_test, target_train, target_test = \
    train_test_split(feat_data, targ_data, test_size=.2, stratify=targ_data)

In [7]:
# Porcentaje de etiquetas en train
pd.Series(target_train).value_counts(normalize=True)

0    0.626374
1    0.373626
dtype: float64

In [8]:
# Porcentaje de etiquetas en test
pd.Series(target_test).value_counts(normalize=True)

0    0.631579
1    0.368421
dtype: float64

## III. EDA automático

In [9]:
comp = sv.compare(pd.DataFrame(data=feat_train, columns=featu_columns), pd.DataFrame(data=feat_test, columns=featu_columns))

                                             |          | [  0%]   00:00 -> (? left)

In [10]:
comp.show_html('test_train_comparaison.html')

Report test_train_comparaison.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## IV. Entrenamiento del modelo

In [11]:
model = lgb.LGBMClassifier(boosting_type="goss", num_leaves=50)
model.fit(feat_train, target_train);

In [12]:
print(classification_report(target_test, model.predict(feat_test)))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97        72
           1       0.97      0.93      0.95        42

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



## V. Local Interpretable Model-agnostic Explanations

In [15]:
explainer = lime.lime_tabular.LimeTabularExplainer(feat_train, feature_names=featu_columns, discretize_continuous=True)

In [19]:
i = 3

In [20]:
exp = explainer.explain_instance(feat_test[i], model.predict_proba, num_features=5)

In [21]:
exp.show_in_notebook(show_table=True, show_all=True)

## Recursos

[Shap](https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d)

[GitHub - Lime](https://github.com/marcotcr/lime)

[Lime](https://towardsdatascience.com/decrypting-your-machine-learning-model-using-lime-5adc035109b5)

[Automated EDA 1](https://analyticsindiamag.com/tips-for-automating-eda-using-pandas-profiling-sweetviz-and-autoviz-in-python/)

[Automated EDA 2](https://towardsdatascience.com/sweetviz-automated-eda-in-python-a97e4cabacde)